# Setup

In [1]:
try:
  import pygeos
except ModuleNotFoundError as e:
  !pip install pygeos==0.10.2
  import pygeos

In [2]:
try:
  import geopandas as gpd
except ModuleNotFoundError as e:
  !pip install geopandas==0.10.1
  import geopandas as gpd

if gpd.__version__ != "0.10.1":
  !pip install -U geopandas==0.10.1
  import geopandas as gpd

/home/napo/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
try:
  import pyrosm
except ModuleNotFoundError as e:
  !pip install pyrosm==0.6.1
  import pyrosm


# Exercise
- download from OpenStreetMap all supermarkets inside the bounding box of the city in this point<br/>
   latitude: 46.21209<br/>
   longitude: 11.09351
- identify the longest road of the city (state roads, walking routes, motorways are excluded). Please use "unclassified"
- How many drinking water are in this city?
- How many benches in this city have the backrest?


## download from OpenStreetMap all supermarkets inside the bounding box of the city in this point and

   latitude: 46.21209<br/>
   longitude: 11.09351

In [4]:
import geopandas as gpd

In [5]:
latitude = 46.21209
longitude = 11.09351

## find the city

### method 1 - reverse geocoding

In [6]:
from geopy.geocoders import ArcGIS

In [7]:
latlon = str(latitude) + "," + str(longitude);

In [8]:
geolocator = ArcGIS(user_agent="Mozilla/5.0 (Linux; Android 10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Mobile Safari/537.36")

In [9]:
location = geolocator.reverse(latlon)

In [10]:
location.raw

{'Match_addr': 'Via Francesco Filos 12A, 38017, Mezzolombardo, Trento',
 'LongLabel': 'Via Francesco Filos 12A, 38017, Mezzolombardo, Trento, ITA',
 'ShortLabel': 'Via Francesco Filos 12A',
 'Addr_type': 'PointAddress',
 'Type': '',
 'PlaceName': '',
 'AddNum': '12A',
 'Address': 'Via Francesco Filos 12A',
 'Block': '',
 'Sector': '',
 'Neighborhood': '',
 'District': 'Mezzolombardo',
 'City': 'Mezzolombardo',
 'MetroArea': '',
 'Subregion': 'Trento',
 'Region': 'Trentino-Alto Adige',
 'Territory': '',
 'Postal': '38017',
 'PostalExt': '',
 'CountryCode': 'ITA'}

In [11]:
city = location.raw['City']

In [12]:
city

'Mezzolombardo'

### method 2 - spatial relation

In [13]:
from shapely.geometry import Point

In [14]:
point = Point(longitude, latitude)

In [15]:
urlmunicipalities2021 = 'https://github.com/napo/geospatial_course_unitn/raw/master/data/istat/shapefile_istat_municipalities_2021.zip'
municipalities2021 = gpd.read_file(urlmunicipalities2021)

In [16]:
muncipality = municipalities2021[municipalities2021.to_crs(epsg=4326).geometry.contains(point)]

In [17]:
muncipality

,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,PRO_COM,PRO_COM_T,COMUNE,COMUNE_A,CC_UTS,SHAPE_LENG,Shape_Le_1,Shape_Area,geometry
5729,2,4,22,0,22,22117,022117,Mezzolombardo,None,0,19772.366696,19772.224335,1.387899e+07,"POLYGON ((659991.073 5121616.939, 660070.949 5..."


In [18]:
city = muncipality.COMUNE.values[0]

In [19]:
city

'Mezzolombardo'

## download from OpenStreetMap

## find the boundig box of Mezzolombardo

In [20]:
municipality = municipalities2021[municipalities2021.COMUNE==city]

## Donwload the PBF of Mezzolombardo from OSM Estratti

![](https://raw.githubusercontent.com/napo/geospatial_course_unitn/master/images/osm_estratti_mezzolombardo.jpg)

In [21]:
url_download_mezzolombardo_pbf = 'https://osmit-estratti.wmcloud.org/dati/poly/comuni/pbf/022117_Mezzolombardo_poly.osm.pbf'
import urllib.request
urllib.request.urlretrieve(url_download_mezzolombardo_pbf ,"mezzolombardo_osm.pbf")

('mezzolombardo_osm.pbf', <http.client.HTTPMessage at 0x7fefb844d9d0>)

## find all the supermarkets in the area



In [22]:
import pyrosm

![](https://raw.githubusercontent.com/napo/geospatial_course_unitn/master/images/tag_supermarket.png)

In [23]:
osm = pyrosm.OSM("mezzolombardo_osm.pbf") 

In [24]:
custom_filter = {'shop': ['supermarket']}

In [25]:
supermarkets = osm.get_pois(custom_filter=custom_filter)

In [26]:
supermarkets.shape

(6, 19)

In [27]:
supermarkets

,version,timestamp,lon,tags,id,lat,changeset,addr:city,addr:country,addr:housenumber,addr:housename,addr:postcode,addr:street,name,opening_hours,shop,geometry,osm_type,operator
0,1,1409592099,11.089408,None,3054012770,46.213261,0.0,None,None,None,None,None,None,Despar,None,supermarket,POINT (11.08941 46.21326),node,NaN
1,5,1594466282,11.100459,"{""brand"":""EuroSpin"",""level"":""0"",""wheelchair"":""...",4557760211,46.207256,0.0,Mezzolombardo,IT,65/A,Complesso Commerciale Braide,38017,Via Trento,EuroSpin,Mo-Su 08:00-20:00,supermarket,POINT (11.10046 46.20726),node,NaN
2,8,1523179735,NaN,"{""building"":""yes"",""ref:vatin"":""IT00108640228""}",39484297,NaN,NaN,Mezzolombardo,IT,33,NaN,38017,Via Guido Fiorini,Orvea,None,supermarket,"POLYGON ((11.10120 46.20900, 11.10125 46.20917...",way,OR.VE.A. S.P.A.
3,7,1615675726,NaN,"{""building"":""yes""}",73412650,NaN,NaN,Mezzolombardo,IT,5,NaN,38017,Via Francesco Morigl,Amort,None,supermarket,"POLYGON ((11.09448 46.21776, 11.09412 46.21755...",way,None
4,3,1635084049,NaN,"{""brand"":""Spar"",""brand:wikidata"":""Q610492"",""br...",301331720,NaN,NaN,None,None,None,NaN,None,None,Despar,"Mo-Sa 07:30-13:00, 14:30-19:30",supermarket,"POLYGON ((11.08941 46.21320, 11.08951 46.21327...",way,None
5,5,1594466282,NaN,"{""brand"":""Lidl"",""fixme"":""position"",""source"":""l...",567538828,NaN,NaN,Mezzolombardo,IT,79,NaN,38017,Via Alcide De Gasperi,LIDL,Mo-Sa 08:00-21:00; Su 08:00-20:00,supermarket,"POLYGON ((11.09423 46.21056, 11.09453 46.21074...",way,LIDL ITALIA SRL


In [28]:
print("there are %s supermarkets in Mezzolombardo" % supermarkets.shape[0])

there are 6 supermarkets in Mezzolombardo


In [29]:
supermarkets.name

0      Despar
1    EuroSpin
2       Orvea
3       Amort
4      Despar
5        LIDL
Name: name, dtype: object

# identify the longest road of the city


In [30]:
roads = osm.get_network(network_type='all')

In [31]:
roads.columns

Index(['access', 'area', 'bicycle', 'bridge', 'cycleway', 'foot', 'footway',
       'highway', 'junction', 'lanes', 'maxspeed', 'motor_vehicle', 'name',
       'oneway', 'ref', 'service', 'segregated', 'smoothness', 'surface',
       'tracktype', 'tunnel', 'id', 'timestamp', 'version', 'tags', 'osm_type',
       'geometry', 'length'],
      dtype='object')

In [32]:
roads.highway.unique()

array(['tertiary', 'unclassified', 'residential', 'track', 'primary',
       'secondary', 'cycleway', 'tertiary_link', 'path', 'service',
       'primary_link', 'steps', 'footway', 'pedestrian', 'secondary_link'],
      dtype=object)

![](https://github.com/napo/geospatial_course_unitn/raw/master/images/tag_highways.gif)

## lenght of unclassified and residential togheter

In [33]:
roads_unclassified_residential = roads[(roads.highway=='unclassified') |  (roads.highway == 'tertiary')]

In [34]:
roads_unclassified_residential

,access,area,bicycle,bridge,cycleway,foot,footway,highway,junction,lanes,...,surface,tracktype,tunnel,id,timestamp,version,tags,osm_type,geometry,length
0,None,None,None,None,None,None,None,tertiary,roundabout,None,...,None,None,None,24732926,1598641256,7,None,way,"MULTILINESTRING ((11.10439 46.19333, 11.10438 ...",132.0
1,None,None,None,None,None,None,None,tertiary,roundabout,None,...,None,None,None,24732928,1464199823,9,None,way,"MULTILINESTRING ((11.10043 46.20075, 11.10048 ...",100.0
2,None,None,None,None,None,None,None,tertiary,roundabout,None,...,None,None,None,24732930,1598641256,10,None,way,"MULTILINESTRING ((11.10073 46.20638, 11.10078 ...",106.0
3,None,None,None,None,None,None,None,tertiary,None,None,...,None,None,None,24732932,1464199684,9,"{""loc_ref"":""SP90""}",way,"MULTILINESTRING ((11.10087 46.20666, 11.10091 ...",108.0
4,None,None,None,None,None,None,None,unclassified,None,None,...,None,None,None,25413543,1522001556,26,None,way,"MULTILINESTRING ((11.09512 46.20965, 11.09569 ...",786.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,None,None,None,None,None,None,None,tertiary,None,None,...,None,None,None,737863922,1571833099,1,None,way,"MULTILINESTRING ((11.10378 46.19349, 11.10384 ...",16.0
874,None,None,None,None,None,None,None,tertiary,None,None,...,None,None,None,737863923,1571833099,1,None,way,"MULTILINESTRING ((11.10395 46.19349, 11.10378 ...",13.0
897,None,None,None,None,None,None,None,unclassified,None,None,...,None,None,None,915122844,1615309919,1,None,way,"MULTILINESTRING ((11.09157 46.20730, 11.09160 ...",148.0
927,None,None,None,None,None,None,None,unclassified,None,None,...,sett,None,None,915122880,1615309919,1,None,way,"MULTILINESTRING ((11.09247 46.20816, 11.09265 ...",106.0


In [35]:
roads_unclassified_residential['name'].value_counts()

Via Trento                   4
Via Carlo Devigili           3
Via Sant'Antonio             3
Piazza Pio XII               3
Via Rotaliana                2
Via della Rupe               2
Piazza Cassa di Risparmio    2
Via Milano                   1
Via Guido Fiorini            1
Piazza Cesare Battisti       1
Corso del Popolo             1
Via Francesco Filos          1
Via degli Alpini             1
Viale Fenice                 1
Via Fabio Filzi              1
Località Galletta            1
Name: name, dtype: int64

In [36]:
names = roads_unclassified_residential.name.unique()

In [37]:
names

array([None, 'Via Carlo Devigili', 'Via Rotaliana', 'Via Trento',
       "Via Sant'Antonio", 'Via Milano', 'Piazza Pio XII',
       'Via della Rupe', 'Via Guido Fiorini', 'Piazza Cassa di Risparmio',
       'Piazza Cesare Battisti', 'Corso del Popolo',
       'Via Francesco Filos', 'Via degli Alpini', 'Viale Fenice',
       'Via Fabio Filzi', 'Località Galletta'], dtype=object)

In [38]:
roads_lenght = {}
rodas_in_meters = roads_unclassified_residential.to_crs(epsg=32632)
for name in names:
  road = rodas_in_meters[rodas_in_meters.name==name]
  road_lenght = road.length.sum()
  roads_lenght[name] = road_lenght


In [39]:
roads_lenght

{None: 0.0,
 'Via Carlo Devigili': 936.754889705999,
 'Via Rotaliana': 774.3678080999027,
 'Via Trento': 1479.2922178941103,
 "Via Sant'Antonio": 317.20741812669604,
 'Via Milano': 428.4259989016135,
 'Piazza Pio XII': 244.72762778938096,
 'Via della Rupe': 657.2358490220092,
 'Via Guido Fiorini': 262.2489738895878,
 'Piazza Cassa di Risparmio': 101.69215459438668,
 'Piazza Cesare Battisti': 74.36442381488386,
 'Corso del Popolo': 254.15957956408997,
 'Via Francesco Filos': 101.21436392814823,
 'Via degli Alpini': 223.4884323682712,
 'Viale Fenice': 377.86898287344684,
 'Via Fabio Filzi': 131.1560732099417,
 'Località Galletta': 321.34567496279004}

In [40]:
max(roads_lenght, key=roads_lenght.get)

'Via Trento'

In [41]:
longest_road = max(roads_lenght, key=roads_lenght.get)

In [42]:
roads_lenght[longest_road]

1479.2922178941103

In [43]:
print("the longest road (unclassified + residential) in Mezzolombardo is %s and is long %.1f km" % (str(longest_road),roads_lenght[longest_road]/1000))

the longest road (unclassified + residential) in Mezzolombardo is Via Trento and is long 1.5 km


## the case only with unclassified

In [44]:
roads_unclassified = roads[(roads.highway=='unclassified')] 

In [45]:
names = roads_unclassified.name.unique()

In [46]:
roads_lenght = {}
rodas_in_meters = roads_unclassified.to_crs(epsg=32632)
for name in names:
  road = rodas_in_meters[rodas_in_meters.name==name]
  road_lenght = road.length.sum()
  roads_lenght[name] = road_lenght

In [47]:
longest_road = max(roads_lenght, key=roads_lenght.get)

In [48]:
print("the longest road (unclassified) is %s and is long %.1f km" % (str(longest_road),roads_lenght[longest_road]/1000))

the longest road (unclassified) is Via Trento and is long 1.5 km


## where is this road?
... we can use overpass-turbo.eu with this wizard query

```
name='Via Rotaliana' in Mezzolombardo
```

http://overpass-turbo.eu/s/1cuk

![](https://github.com/napo/geospatial_course_unitn/raw/master/images/roads_lenght_mezzolombardo.gif)

# How many drinking water are in this city?

In [49]:
custom_filter = {'amenity': ['drinking_water']}

In [50]:
drinking_water = osm.get_pois(custom_filter=custom_filter)

In [51]:
drinking_water.amenity.unique()

array(['drinking_water'], dtype=object)

In [52]:
drinking_water.shape

(17, 10)

In [53]:
drinking_water.head(4)

,version,timestamp,lon,tags,id,lat,changeset,amenity,geometry,osm_type
0,2,1489783071,11.092862,None,388416004,46.225494,0,drinking_water,POINT (11.09286 46.22549),node
1,2,1489783071,11.094058,None,388416008,46.224617,0,drinking_water,POINT (11.09406 46.22462),node
2,1,1282084067,11.089766,None,865333800,46.214092,0,drinking_water,POINT (11.08977 46.21409),node
3,3,1306775522,11.093194,None,937561830,46.213287,0,drinking_water,POINT (11.09319 46.21329),node


In [54]:
drinking_water.explore()

# How many benches in this city have the backrest?

In [55]:
custom_filter = {'amenity': ['bench']}

In [56]:
benchs = osm.get_pois(custom_filter=custom_filter)

In [57]:
benchs.shape

(71, 10)

In [58]:
benchs.columns

Index(['version', 'timestamp', 'lon', 'tags', 'id', 'lat', 'changeset',
       'amenity', 'geometry', 'osm_type'],
      dtype='object')

In [59]:
benchs.tags.unique()

array(['{"backrest":"yes"}', '{"backrest":"no"}',
       '{"seats":"5","backrest":"yes"}', '{"seats":"4","backrest":"yes"}',
       None], dtype=object)

quick&dirty solution

In [60]:
benchs_with_backrest = benchs[benchs.tags.str.find('"backrest":"yes"') > -1]

In [61]:
benchs_with_backrest.tags.unique()

array(['{"backrest":"yes"}', '{"seats":"5","backrest":"yes"}',
       '{"seats":"4","backrest":"yes"}'], dtype=object)

In [62]:
benchs_with_backrest.shape

(62, 10)

In [63]:
print("the total of benchs tagged with the presence of a backrest is %s" % (str(benchs_with_backrest.shape[0])))

the total of benchs tagged with the presence of a backrest is 62
